In [2]:
import pandas as pd
import json
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Sisend ja väljundfail
INPUT_FILE = "data/toorandmed_aasta.csv"
OUTPUT_FILE = "data/andmed.csv"

df = pd.read_csv(INPUT_FILE, low_memory=False)

df.head(2)

# NB! Notebooki jooksutamisel puhastatakse andmed. Kui kuskilt keskelt suvalisest kohast midagi jooksutama hakta, siis lähevad andmed katki.
# Jooksutage kogu faili korraga või alati algusest lõpuni.

,course_uuid,code,title__en,version__title__en,credits,overview__description__en,version__overview__description__en,overview__learning_outcomes_text_en,uuid,state__code,state__et,state__en,last_update,title__et,general__input_languages,general__structural_unit_shares,general__type__code,general__type__et,general__type__en,general__structural_units,additional_info__duration_in_semesters,additional_info__assessment_scale__code,additional_info__assessment_scale__et,additional_info__assessment_scale__en,additional_info__is_vota_course,additional_info__is_continuous_learning_course,overview__description__et,overview__objectives,overview__learning_outcomes,_actions__is_editable,_actions__is_version_creatable,_actions__is_deletable,overview__objectives_text_en,version__uuid,version__code,version__state__code,version__state__et,version__state__en,version__last_update,version__title__et,version__credits,version__credit_type,version__target__year__code,version__target__year__et,version__target__year__en,version__target__semester__code,version__target__semester__et,version__target__semester__en,version__target__study_type__code,version__target__study_type__et,version__target__study_type__en,version__target__language__code,version__target__language__et,version__target__language__en,version__target__part__code,version__target__part__et,version__target__part__en,version__target__faculty__code,version__target__faculty__name__en,version__target__faculty__name__et,version__target__faculty__address,version__target__faculty__city,version__target__faculty__level,version__target__faculty__academic,version__target__faculty__supports_continuous_learning,version__target__course_main_structural_unit__code,version__target__course_main_structural_unit__name__en,version__target__course_main_structural_unit__name__et,version__target__course_main_structural_unit__webpage_url,version__target__course_main_structural_unit__address,version__target__course_main_structural_unit__city,version__target__course_main_structural_unit__street,version__target__course_main_structural_unit__zip,version__target__course_main_structural_unit__level,version__target__course_main_structural_unit__academic,version__target__course_main_structural_unit__supports_continuous_learning,version__target__course_main_structural_unit__parent_code,version__target__label__en,version__target__label__et,version__general__year__code,version__general__year__et,version__general__year__en,version__general__input_languages,version__general__structural_unit_shares,version__general__type__code,version__general__type__et,version__general__type__en,version__additional_info__is_vota_course,version__additional_info__is_continuous_learning_course,version__additional_info__study_levels,version__additional_info__hours__lecture,version__additional_info__hours__practice,version__additional_info__hours__seminar,version__additional_info__hours__colloquium,version__additional_info__hours__web_learning,version__additional_info__hours__independent_work,version__additional_info__hours__individual_work,version__additional_info__hours__internship,version__additional_info__is_enlight_course,version__overview__study_languages,version__overview__description__et,version__overview__objectives,version__overview__learning_outcomes,version__grading__independent_work_assessments,version__grading__grade_preconditions__et,version__grading__grade_evaluation__et,version__grading__debt_elimination__et,version__grading__assessment_scale__code,version__grading__assessment_scale__et,version__grading__assessment_scale__en,version__resources__web_based__code,version__resources__web_based__et,version__resources__web_based__en,version__resources__website_url,version__resources__is_moodle_connected,version__resources__learning_environments,version__resources__materials,version__participants__lecturers,version__schedule__weeks__et,version__schedule__entries,version__parent_uuid,version__parent_code,version__parent_credits,version___action

In [3]:

# Kasulik info
used_columns = ["code", 
                "credits", 
                "additional_info__prerequisites", 
                "overview__description__en", 
                "overview__description__et",
                "title__en", 
                "title__et",
                "version__target__semester__et", 
                "additional_info__assessment_scale__et", 
                "version__overview__study_languages",
                "version__target__course_main_structural_unit__city", 
                "version__additional_info__study_levels", 
                "version__grading__grade_evaluation__et", 
                "overview__learning_outcomes", 
                "overview__objectives"]

df = df[used_columns]
df.head()

,code,credits,additional_info__prerequisites,overview__description__en,overview__description__et,title__en,title__et,version__target__semester__et,additional_info__assessment_scale__et,version__overview__study_languages,version__target__course_main_structural_unit__city,version__additional_info__study_levels,version__grading__grade_evaluation__et,overview__learning_outcomes,overview__objectives
0,OIEO.06.046,6.00,NaN,"The course covers the main topics of private international law (PIL), its terminology and develo...","Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunemist,...",Private International Law,Rahvusvaheline eraõigus,kevad,"Eristav (A, B, C, D, E, F, mi)","[{""code"": ""et"", ""en"": ""Estonian"", ""et"": ""eesti keel""}]",Tartu linn,"[{""code"": ""master"", ""en"": ""master's studies"", ""et"": ""magistriõpe""}]",100% kirjalik eksam.,"[{""en"": ""At the end of the course the student is able to:"", ""et"": ""Kursuse lõpuks peab üliõpilan...","[{""en"": ""The purpose of the course is to teach the theoretical principles of private internation..."
1,KKSB.05.092,3.00,NaN,Subject Practicing in the work environment the students expand their knowledge in the field of p...,Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkonnas ...,Practice in the Work Environment,Praktiseerimine töökeskkonnas,kevad,"Eristamata (arv, m.arv, mi)","[{""code"": ""et"", ""en"": ""Estonian"", ""et"": ""eesti keel""}]",Tartu linn,"[{""code"": ""bachelor"", ""en"": ""bachelor's studies"", ""et"": ""bakalaureuseõpe""}]",NaN,"[{""en"": ""Student:"", ""et"": ""Õpilane:""}, {""en"": ""understands and follows the work ethic and organi...","[{""en"": ""The aim of practice is gaining work experience in the work environment"", ""et"": ""Aine ee..."
2,ARKI.02.030,2.25,"[{""code"": ""AR00.00.016"", ""credits"": 17, ""last_update"": ""2024-02-05T11:04:36"", ""required"": true, ...","Diagnostic and treatment principles, methods and tactics of surgical diseases in children. The c...","Lastekirurgiliste haiguste diagnoosimise ja ravi printsiibid, metoodika ja taktika. Antud aine m...",Paediatric Surgery,Lastekirurgia,sügis,"Eristamata (arv, m.arv, mi)","[{""code"": ""en"", ""en"": ""English"", ""et"": ""inglise keel""}]",Tartu linn,"[{""code"": ""bachelor_master"", ""en"": ""integrated bachelor's and master's studies"", ""et"": ""integree...","A part of the complec exam ""Surgery""","[{""en"": ""After completing the course the student knows"", ""et"": ""Õppe tulemusena peavad üliõpilas...","[{""en"": ""The aims of the course are: to provide theoretical knowledge of the basics of paediatri..."
3,LOFY.04.041,6.00,"[{""alternatives"": [{""code"": ""LTFY.04.001"", ""credits"": 5, ""last_update"": ""2024-02-05T11:04:36"", ""...",In this lecture series we give a systematic introduction to quantum field theory. On the basis o...,Loengukursuses esitatakse süstemaatiline sissejuhatus kvantväljateooriasse. Lähtudes variatsioon...,Quantum Field Theory,Kvantväljateooria,kevad,"Eristav (A, B, C, D, E, F, mi)","[{""code"": ""et"", ""en"": ""Estonian"", ""et"": ""eesti keel""}]",NaN,"[{""code"": ""master"", ""en"": ""master's studies"", ""et"": ""magistriõpe""}, {""code"": ""doctoral"", ""en"": ""...",Hinde kujunemine: \n- kodused ülesanded\n- eksam,"[{""en"": ""After the course, the student:"", ""et"": ""Aine läbinud õpilane:""}, {""en"": ""1. knows the s...","[{""en"": ""The aim of this course is to introduce quantum fields and their perturbation theory via..."
4,ARSK.01.030,2.25,NaN,"For Students Gives General Practical Knowledge in Ethiopathogenesis, Diagnosis, Treatment and Pr...","Anda üliõpilastele üldised praktilised teadmised endokrinoloogiliste haiguste etiopatogeneesist,...",Endocrinology,Endokrinoloogia,kevad,"Eristav (A, B, C, D, E, F, mi)","[{""code"": ""en"", ""en"": ""English"", ""et"": ""inglise keel""}]",Tartu linn,"[{""code"": ""bachelor_master"", ""en"": ""integrated bachelor's 

### Andmete puhastmine
5 veergu on JSON formaadis, need teen kaheks uueks veeruks (eesti ja inglise keele jaoks). Teised veerud on kõik juba korras ja neid ma ei puhasta hetkel.

In [4]:
# JSON formaadis on:
# additional_info__prerequisites
# version__overview__study_languages
# version__additional_info__study_levels
# overview__learning_outcomes
# overview__objectives

def parse_json_safe(json_str):
    """
    Teisendab JSON-stringi turvaliselt Pythoni objektiks (list või dict).
    Käsitleb tühje väärtusi (NaN, None) ja katkist JSON-it, tagastades vea korral None.
    """
    if json_str is None or (isinstance(json_str, float) and pd.isna(json_str)):
        return np.nan
    try:
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError):
        return None
    

# Eeldusainete veerg kaheks veeruks: prereq_en ja prereq_et
# funktsioon muudab JSON formaadi õigele kujule ja tagastab andmed kahes keeles eraldi
def extract_prerequisites(prereq_list):
    if prereq_list is None or (isinstance(prereq_list, float) and pd.isna(prereq_list)):
        return np.nan, np.nan
    if not isinstance(prereq_list, list):
        return np.nan, np.nan
    
    en_list = []
    et_list = []
    
    for course in prereq_list:
    
        if not isinstance(course, dict):
            continue
        
        code = course.get("code")
        credits = course.get("credits")
        title = course.get("title", {})
        
        title_en = title.get("en")
        title_et = title.get("et")
        
        if code and credits and title_en and title_et:
            en_list.append(f"{code} - {title_en} ({credits} ECTS)")
            et_list.append(f"{code} - {title_et} ({credits} EAP)")
    
    if not en_list:
        return np.nan, np.nan
    
    return "; ".join(en_list), "; ".join(et_list)

# uute veergude loomine 
df["additional_info__prerequisites"] = df["additional_info__prerequisites"].apply(parse_json_safe)
df[["prereq_en", "prereq_et"]] = df["additional_info__prerequisites"].apply(
    lambda x: pd.Series(extract_prerequisites(x))
)

In [5]:

# Sama ideega funktsioon keele veeru jaoks
def extract_languages(lang_list):
    if lang_list is None or (isinstance(lang_list, float) and pd.isna(lang_list)):
        return np.nan, np.nan
    if not isinstance(lang_list, list):
        return np.nan, np.nan

    en_vals = []
    et_vals = []

    for lang in lang_list:
        if not isinstance(lang, dict):
            continue

        en_name = lang.get("en")
        et_name = lang.get("et")

        if en_name:
            en_vals.append(en_name)
        if et_name:
            et_vals.append(et_name)

    if not en_vals and not et_vals:
        return np.nan, np.nan

    return "; ".join(en_vals), "; ".join(et_vals)

# uued veerud
df["version__overview__study_languages"] = df["version__overview__study_languages"].apply(parse_json_safe)
df[["study_language_en", "study_language_et"]] = df["version__overview__study_languages"].apply(
    lambda x: pd.Series(extract_languages(x))
)

In [6]:

# õppeastmete jaoks
def extract_study_levels(level_list):
    if level_list is None or (isinstance(level_list, float) and pd.isna(level_list)):
        return np.nan, np.nan
    if not isinstance(level_list, list):
        return np.nan, np.nan

    en_vals = []
    et_vals = []

    for lvl in level_list:
        if not isinstance(lvl, dict):
            continue

        en_name = lvl.get("en")
        et_name = lvl.get("et")

        if en_name:
            en_vals.append(en_name)
        if et_name:
            et_vals.append(et_name)

    if not en_vals and not et_vals:
        return np.nan, np.nan

    return "; ".join(en_vals), "; ".join(et_vals)

# uued veerud
df["version__additional_info__study_levels"] = df["version__additional_info__study_levels"].apply(parse_json_safe)
df[["study_level_en", "study_level_et"]] = df["version__additional_info__study_levels"].apply(
    lambda x: pd.Series(extract_study_levels(x))
)

In [7]:

# õpitulemuste jaoks
def extract_learning_outcomes(lo_list):
    if lo_list is None or (isinstance(lo_list, float) and pd.isna(lo_list)):
        return np.nan, np.nan
    if not isinstance(lo_list, list):
        return np.nan, np.nan

    en_parts = []
    et_parts = []

    for item in lo_list:
        if not isinstance(item, dict):
            continue

        en_text = item.get("en")
        et_text = item.get("et")

        if en_text:
            clean_en = en_text.replace("\t", " ").strip()
            en_parts.append(clean_en)

        if et_text:
            clean_et = et_text.replace("\t", " ").strip()
            et_parts.append(clean_et)

    if not en_parts and not et_parts:
        return np.nan, np.nan

    return "\n".join(en_parts), "\n".join(et_parts)

# uued veerud
df["overview__learning_outcomes"] = df["overview__learning_outcomes"].apply(parse_json_safe)
df[["learning_outcomes_en", "learning_outcomes_et"]] = df["overview__learning_outcomes"].apply(
    lambda x: pd.Series(extract_learning_outcomes(x))
)

In [8]:

# õpieesmärkide jaoks
def extract_course_objectives(obj_list):
    if obj_list is None or (isinstance(obj_list, float) and pd.isna(obj_list)):
        return np.nan, np.nan
    if not isinstance(obj_list, list):
        return np.nan, np.nan

    en_parts, et_parts = [], []

    for item in obj_list:
        if not isinstance(item, dict):
            continue

        en_text = item.get("en")
        et_text = item.get("et")

        if en_text:
            en_parts.append(" ".join(en_text.replace("\t", " ").split()))
        if et_text:
            et_parts.append(" ".join(et_text.replace("\t", " ").split()))

    if not en_parts and not et_parts:
        return np.nan, np.nan

    # objectives are usually 1 paragraph -> space join is fine
    return " ".join(en_parts), " ".join(et_parts)

# uued veerud
df["overview__objectives"] = df["overview__objectives"].apply(parse_json_safe)
df[["overview_objective_en", "overview_objective_et"]] = df["overview__objectives"].apply(
    lambda x: pd.Series(extract_course_objectives(x))
)

In [9]:
# Kustutada algsed üleliigsed veerud
df = df.drop(columns=["additional_info__prerequisites", "version__overview__study_languages", "version__additional_info__study_levels",
                     "overview__learning_outcomes", "overview__objectives"])

In [10]:
final_columns = ["code",
                 "credits",
                 "overview__description__en",
                 "overview__description__et",
                 "title__en",
                 "title__et",
                 "version__target__semester__et",
                 "additional_info__assessment_scale__et",
                 "version__target__course_main_structural_unit__city",
                 "version__grading__grade_evaluation__et",
                 "prereq_en",
                 "prereq_et",
                 "study_language_en",
                 "study_language_et",
                 "study_level_en",
                 "study_level_et",
                 "learning_outcomes_en",
                 "learning_outcomes_et",
                 "overview_objective_en",
                 "overview_objective_et"]

existing_cols = [c for c in final_columns if c in df.columns]
df_final = df[existing_cols].copy()

# Kirjelduse veeru tegemine. See on sõnastiku kujul. Võti on veerumini ja väärtus on selle rea ja veeru väärtus.
df_final["kirjeldus"] = df_final.to_dict(orient="records")

print(f"Lõplik ridade arv: {len(df_final)}, lõplik veergude arv: {len(df_final.columns)}")
df_final.head()

Lõplik ridade arv: 3156, lõplik veergude arv: 21


,code,credits,overview__description__en,overview__description__et,title__en,title__et,version__target__semester__et,additional_info__assessment_scale__et,version__target__course_main_structural_unit__city,version__grading__grade_evaluation__et,prereq_en,prereq_et,study_language_en,study_language_et,study_level_en,study_level_et,learning_outcomes_en,learning_outcomes_et,overview_objective_en,overview_objective_et,kirjeldus
0,OIEO.06.046,6.00,"The course covers the main topics of private international law (PIL), its terminology and develo...","Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunemist,...",Private International Law,Rahvusvaheline eraõigus,kevad,"Eristav (A, B, C, D, E, F, mi)",Tartu linn,100% kirjalik eksam.,NaN,NaN,Estonian,eesti keel,master's studies,magistriõpe,At the end of the course the student is able to:\n- explain the nature of the main institutes of...,Kursuse lõpuks peab üliõpilane olema võimeline:\n- selgitama rahvusvahelise eraõiguse põhiinstit...,The purpose of the course is to teach the theoretical principles of private international law (P...,Õppeaine üldiseks eesmärgiks on õpetada rahvusvahelise eraõiguse teoreetilisi alusteadmisi; selg...,"{'code': 'OIEO.06.046', 'credits': 6.0, 'overview__description__en': 'The course covers the main..."
1,KKSB.05.092,3.00,Subject Practicing in the work environment the students expand their knowledge in the field of p...,Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkonnas ...,Practice in the Work Environment,Praktiseerimine töökeskkonnas,kevad,"Eristamata (arv, m.arv, mi)",Tartu linn,NaN,NaN,NaN,Estonian,eesti keel,bachelor's studies,bakalaureuseõpe,Student:\nunderstands and follows the work ethic and organization in the work environment;;\ncan...,Õpilane:\nmõistab ja järgib töökeskkonnas tööeetikat ja-korraldust;\noskab teha meeskonnatööd ni...,The aim of practice is gaining work experience in the work environment,Aine eesmärgiks on töökogemuse omandamine töökeskkonnas,"{'code': 'KKSB.05.092', 'credits': 3.0, 'overview__description__en': 'Subject Practicing in the ..."
2,ARKI.02.030,2.25,"Diagnostic and treatment principles, methods and tactics of surgical diseases in children. The c...","Lastekirurgiliste haiguste diagnoosimise ja ravi printsiibid, metoodika ja taktika. Antud aine m...",Paediatric Surgery,Lastekirurgia,sügis,"Eristamata (arv, m.arv, mi)",Tartu linn,"A part of the complec exam ""Surgery""",AR00.00.016 - Fundamentals of Diagnostics (17 ECTS),AR00.00.016 - Diagnostika alused (17 EAP),English,inglise keel,integrated bachelor's and master's studies,integreeritud bakalaureuse- ja magistriõpe,"After completing the course the student knows\n1) the epidemiology, etiology, pathogenesis and p...",Õppe tulemusena peavad üliõpilased teadma:\n1) sagedamini esinevate lastekirurgiliste haiguste e...,The aims of the course are: to provide theoretical knowledge of the basics of paediatric surgery...,"Anda teoreetilisi teadmisi lastekirurgia alustest (laste üldkirurgia, vastsündinute kirurgia, on...","{'code': 'ARKI.02.030', 'credits': 2.25, 'overview__description__en': 'Diagnostic and treatment ..."
3,LOFY.04.041,6.00,In this lecture series we give a systematic introduction to quantum field theory. On the basis o...,Loengukursuses esitatakse süstemaatiline sissejuhatus kvantväljateooriasse. Lähtudes variatsioon...,Quantum Field Theory,Kvantväljateooria,kevad,"Eristav (A, B, C, D, E, F, mi)",NaN,Hinde kujunemine: \n- kodused ülesanded\n- eksam,LOFY.04.073 - Quantum Mechanics (6 ECTS),LOFY.04.073 - Kvantmehaanika (6 EAP),Estonian,eesti keel,master's studies; doctoral studies,magistriõpe; doktoriõpe,"After the course, the student:\n1. knows the special features of second quantisation.\n2. is abl...",Aine läbinud õpilane:\n1. tunneb teise kvantiseerimise omapärasid.\n2. oskab arvutada teistkvant...,The aim of this course is to introduce quantum fields and their perturbation theory vi

In [11]:
# Puuduvate väärtuste arv
df_final.isna().sum()

code                                                     0
credits                                                  0
overview__description__en                                0
overview__description__et                                0
title__en                                                0
title__et                                                0
version__target__semester__et                           21
additional_info__assessment_scale__et                    0
version__target__course_main_structural_unit__city    1169
version__grading__grade_evaluation__et                 390
prereq_en                                             2590
prereq_et                                             2590
study_language_en                                       22
study_language_et                                       22
study_level_en                                          60
study_level_et                                          60
learning_outcomes_en                                    

In [12]:

# Enimlevinud väärtused kategoorilistes veergudes
categorical_features = [
                        "version__target__semester__et",
                        "additional_info__assessment_scale__et",
                        "version__target__course_main_structural_unit__city",
                        "version__grading__grade_evaluation__et",
                        "prereq_et",
                        "study_language_et",
                        "study_level_et",
                        ]

for col in categorical_features:
    print(f"\n=== {col} ===")
    print(df_final[col].value_counts().head())


=== version__target__semester__et ===
version__target__semester__et
kevad    1570
sügis    1565
Name: count, dtype: int64

=== additional_info__assessment_scale__et ===
additional_info__assessment_scale__et
Eristav (A, B, C, D, E, F, mi)    1863
Eristamata (arv, m.arv, mi)       1291
Kaitsmine                            2
Name: count, dtype: int64

=== version__target__course_main_structural_unit__city ===
version__target__course_main_structural_unit__city
Tartu linn         1689
Narva linn          129
Viljandi linn       119
Pärnu linn           34
Tõravere alevik      11
Name: count, dtype: int64

=== version__grading__grade_evaluation__et ===
version__grading__grade_evaluation__et
Kirjalik eksam                                                                                                  10
Ettekande koostamine ja esitlemine seminaril. Oponeerimine. Modereerimine. Aktiivne osalemine diskussioonis.     5
Lõpphinne kujuneb kõikide praktikaülesannete täitmise ning juhendajate ja e

In [13]:
# kirjeldus valja keskmine pikkus on 3231 tähemärki?
df_final["kirjeldus"].astype(str).str.len().describe()

count     3156.000000
mean      3231.090621
std       1214.895333
min        683.000000
25%       2406.000000
50%       3012.000000
75%       3830.000000
max      10123.000000
Name: kirjeldus, dtype: float64

In [14]:
# Sissejuhatus andmeteadusesse informatsioon
mask = df_final["title__et"].str.contains(
    "sissejuhatus andmeteadusesse",
    case=False,
    na=False
)

r = df_final.loc[mask].iloc[0]
print(json.dumps(r.drop(labels=["kirjeldus"]).to_dict(), indent=2, ensure_ascii=False))

{
  "code": "LTAT.02.002",
  "credits": 6.0,
  "overview__description__en": "This course gives a brief overview of the basic concepts, principles and practice of data science. The main goal is to learn to plan and carry out a simple practical data science project. The course covers the main methods for descriptive data analysis and visualization, frequent pattern mining, cluster analysis, principal components analysis, common methods of machine learning for classification and regression (including deep neural networks), managing data and interpreting results of statistical tests. The main stages of data science projects are dicussed and available software tools reviewed. The homeworks are to be solved using the programming language Python 3 and its libraries.",
  "overview__description__et": "Kursus annab lühiülevaate andmeteaduse põhimõistetest, põhiprintsiipidest ning praktilistest aspektidest. Lõppeesmärk on õppida planeerima ning läbi viima lihtsat praktilist andmeteaduse projekti.

In [15]:
# salvesta fail
df_final.to_csv(OUTPUT_FILE, index=False)